## 케라스 모델과 레이어를 맞춤형으로 사용하기
##### url: https://www.tensorflow.org/guide/keras/custom_layers_and_models

- 목차
    - 설정
    - 레이어 클래스
        - 레이어에서 가중치와 연산 캡슐화하기
        - 모범 예제: 입력 차원이 알려지기 전까지 가중치 생성 지연
        - 재귀적으로 레이어 구성하기
        - 데이터가 모델을 통과하는 동안 생성되는 손실 값을 재귀적으로 수집하는 레이어
        - 선택적으로 레이어 직렬화 활성화하기
        - Call 메소드가 가지는 훈련 인자에 대한 특권
    - 모델 작성하기
        - 모델 클래스
        - 하나로 합치기: 엔드 투 엔드 예제
        - 객체 지향 개발 기법을 넘어서: 함수형 API
        
### 설정

In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.keras.backend.clear_session() # 노트북의 상태를 쉽게 초기화할 수 있습니다.

## 레이어 클래스
### 레이어에서 가중치와 연산 캡슐화하기
사용하게 될 주요 데이터구조는 `레이어` 입니다. 레이어는 상태(레이어의 가중치,'weight')와 입력을 출력으로 변환하는 것(호출,'call', 레이어의 순전파)을 캡슐화합니다.<br><br>
여기 밀집되어 연결된 레이어가 있습니다. 이것들은 변수 `w`와 `b`로 나타내어지는 상태를 가지고 있습니다. 

In [9]:
from tensorflow.keras import layers

class Linear(layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                                  dtype='float32'),
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                                  dtype='float32'),
                             trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.06976108 -0.02438297 -0.06618214  0.0508681 ]
 [-0.06976108 -0.02438297 -0.06618214  0.0508681 ]], shape=(2, 4), dtype=float32)


`w`와 `b`가 자동적으로 레이어 속성 집합으로써 추적된다는 것을 확인해보세요.

In [10]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

또한, `add_weight` 메소드를 사용하여 레이어가 가진 가중치에 더 빠르게 접근할 수 있습니다.

In [11]:
class Linear(layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(shape=(units,),
                                initializer='zeros',
                                trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
s = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.03956851  0.10698998 -0.07351424 -0.05690815]
 [-0.03956851  0.10698998 -0.07351424 -0.05690815]], shape=(2, 4), dtype=float32)


#### 레이어는 학습할 수 없는 가중치를 가질 수 있습니다. 
학습 가능한 가중치들 사이에서, 레이어에 학습할 수 없는 가중치들을 추가할 수 있습니다.<br>
레이어를 학습하는 과정에서 역전파 시 이러한 가중치들은 고려하지 않아야 합니다.<br><br>
어떻게 학습할 수 없는 가중치를 더하고 사용하는지 살펴보겠습니다.

In [12]:
class ComputeSum(layers.Layer):
    
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
                                 trainable=False)
        
    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total
    
x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

[2. 2.]
[4. 4.]


해당 가중치들은 `layer.weights`의 일부분이지만, 학습할 수 없는 가중치로 분류됩니다.

In [13]:
print('weights:', len(my_sum.weights))
print('non-trainable weights:', len(my_sum.non_trainable_weights))

# 학습할 수 있는 가중치가 포함되어 있지 않습니다.
print('trainable_weights:', my_sum.trainable_weights)

weights: 1
non-trainable weights: 1
trainable_weights: []
